In [1]:
import os as os
import  pandas as pd
import glob
import re as re
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:

import xarray as xr

In [3]:
datadir='/storage/pdo/pdrmip/regridded/T42'

In [4]:
fils=[]
for (dirpath, dirnames, filenames) in os.walk(datadir):
    fils += [os.path.join(dirpath, file) for file in filenames]

In [133]:
mdls = os.listdir(datadir)

In [134]:
mdls.remove('NCAR-CESM1-CAM4')

In [135]:
df = pd.DataFrame(columns = ['Variable', 'Type', 'Model','Experiment','Configuration','Resolution','period','Filename'])
for j,dp in enumerate(mdls):
    dirpath=datadir+'/'+mdls[j]+'/coupled'
    if os.path.isdir(dirpath):
        #fils=glob.glob(dirpath+'/*.nc')
        tmpl=list(os.walk(dirpath, followlinks=False))
        fils=tmpl[0][2]
        for i,fl in enumerate(fils):
            if Path(tmpl[0][0]+'/'+fl).is_symlink():
                1
            else:
                tmp=re.sub('.*/','',re.sub('.nc','',fl)).split('_')+[tmpl[0][0]+'/'+fl]
                df.loc[len(df)] = tmp

In [154]:
expts=df.Experiment.unique()[:]

In [156]:
vars=['tas','pr'] #df.Variable.unique()

In [158]:
mdls=df.Model.unique()
mdls

array(['CanESM2', 'ECHAM-HAM', 'GISS-E2-R', 'HadGEM2', 'HadGEM3',
       'IPSL-CM5A', 'MIROC-SPRINTARS', 'MPI-ESM', 'NCAR-CESM1-CAM5',
       'NorESM1'], dtype=object)

In [177]:
dsd=dict()

for i,expt in enumerate(expts):
    ldac=[]
    for k,var in enumerate(vars):

        for j,mdl in enumerate(mdls):
            rslt_df = df[(df['Variable'] == var) & (df['Experiment'] == expt) & (df['Model'] == mdl)].groupby('Model').first()

            if mdl in rslt_df.index:
                    #print(rslt_df.loc[mdl].Filename)
                    tmpds=xr.open_dataset(rslt_df.loc[mdl].Filename,decode_times=False)

                    tmp=tmpds[var].squeeze()
                    if 't' in tmp.coords:
                      tmp=tmp.rename({'t': 'time'})

                    if 'time_counter' in tmp.coords:
                      tmp=tmp.rename({'time_counter': 'time'})   
                    nt=len(tmp.coords['time'])
                    tmp.coords['time']=xr.cftime_range(start="1850", periods=nt, freq="M", calendar="noleap")        
                    #print(tmp.coords)
            else:
                tmp=tmp*np.nan
            if j==0:  
                dac=tmp
            else:
                #print('concat')
                dac=xr.concat([dac,tmp],'ens',coords='minimal',compat='override')

        ldac.append(dac)
    ldac=xr.merge(ldac)
    ldac=ldac.groupby('time.year').mean('time').transpose("year", "lat", "lon","ens")
    ldac=ldac.assign_coords({"ens": mdls})
    dsd[expt]=ldac

/uio/kant/div-cicero-u1/bensan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/uio/kant/div-cicero-u1/bensan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/uio/kant/div-cicero-u1/bensan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/uio/kant/div-cicero-u1/bensan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/uio/kant/div-cicero-u1/bensan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **pa

In [182]:
for i,expt in enumerate(expts):
    dsd[expt].to_netcdf('training/'+expt+'_T42_ANN.nc')